In [26]:
import time

import numpy as np
import tensorflow as tf

## Python `for` loop version

In [19]:
tf.reset_default_graph()
# Accumulator Variable
total = tf.Variable(0.0)
# Random inputs to multiply
a = tf.random_normal([1000, 1000])
b = tf.random_normal([1000, 1000])
# Calculate value to add to accumulator variable
mul = tf.matmul(a, b)
mean = tf.reduce_mean(mul)
acc = total.assign_add(mean)
# Initialization op
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    start_t = time.time()
    for i in range(1000):
        final_total = sess.run(acc)
    end_t = time.time()
    print('Time elapsed: {} seconds'.format(end_t - start_t))

Time elapsed: 1.0378661155700684 seconds


## `while_loop` version

In [20]:
tf.reset_default_graph()
# Accumulator Variable
total = tf.Variable(0.0)
i = tf.constant(0)
# Define the loop body
def body(i, _):
    a = tf.random_normal([1000, 1000])
    b = tf.random_normal([1000, 1000])
    mul = tf.matmul(a, b)
    mean = tf.reduce_mean(mul)
    acc = total.assign_add(mean)
    return i+1, acc
# Define the loop condition
def condition(i, _):
    return tf.less(i, 1000)
# Create the while_loop
out = tf.while_loop(condition, body, [i, total])
# Initialization op
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    start_t = time.time()
    final_i, final_total = sess.run(out)
    end_t = time.time()
    print('Time elapsed: {} seconds'.format(end_t - start_t))

Time elapsed: 0.6644983291625977 seconds


## Condensed version of the above

This is the same graph from above, but condensed with by using a `lambda` and some TensorFlow sugar.

In [19]:
tf.reset_default_graph()
# Accumulator Variable
total = tf.Variable(0.0)
# Define the loop body
def body(i, _):
    a = tf.random_normal([1000, 1000])
    b = tf.random_normal([1000, 1000])
    mul = tf.matmul(a, b)
    mean = tf.reduce_mean(mul)
    acc = total.assign_add(mean)
    return i+1, acc
# Create the while_loop
out = tf.while_loop(lambda i, _: i < 1000, body, [0, total])
# Initialization op
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    start_t = time.time()
    final_i, final_total = sess.run(out)
    end_t = time.time()
    print('Time elapsed: {} seconds'.format(end_t - start_t))

0.421863
Time elapsed: 0.6785540000000054 seconds


In [32]:
tf.reset_default_graph()
# Accumulator Variable
total = tf.Variable(0.0)
# Define the loop body
def body(i, _):
    with tf.name_scope('body'):
        a = tf.random_normal([1000, 1000])
        b = tf.random_normal([1000, 1000])
        mul = tf.matmul(a, b)
        mean = tf.reduce_mean(mul)
        acc = total.assign_add(mean)
        return i+1, acc
# Create the while_loop
out = tf.while_loop(lambda i, _: i < 1000, body, [0, total])
# Initialization op
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    start_t = time.time()
    final_i, final_total = sess.run(out)
    end_t = time.time()
    print('Time elapsed: {} seconds'.format(end_t - start_t))
    
tf.summary.FileWriter('tbout/while_loop', graph=tf.get_default_graph()).close()

-0.804292
Time elapsed: 0.7053129999999328 seconds


# 1000 Ops vs 1 `tf.while_loop`

In [24]:
tf.reset_default_graph()
graph_start_time = time.time()
# Random inputs to multiply
a = tf.random_normal([1000, 1000])
for i in range(1000):
    a = tf.matmul(a, tf.random_normal([1000, 1000]))
graph_creation_time = time.time() - graph_start_time
with tf.Session() as sess:
    start_t = time.time()
    final_total = sess.run(a)
    end_t = time.time()
    print('Graph creation time: {} seconds'.format(graph_creation_time))
    print('Time elapsed: {} seconds'.format(end_t - start_t))

Graph creation time: 5.356990337371826 seconds
Time elapsed: 2.4342949390411377 seconds


In [25]:
tf.reset_default_graph()
graph_start_time = time.time()
# Define the loop body
def body(i, a):
    with tf.name_scope('body'):
        return i+1, tf.matmul(a, tf.random_normal([1000, 1000]))
# Create the while_loop
out = tf.while_loop(lambda i, _: i < 1000, body, [0, tf.random_normal([1000, 1000])])
graph_creation_time = time.time() - graph_start_time
with tf.Session() as sess:
    start_t = time.time()
    final_i, final_total = sess.run(out)
    end_t = time.time()
    print('Graph creation time: {} seconds'.format(graph_creation_time))
    print('Time elapsed: {} seconds'.format(end_t - start_t))

Graph creation time: 0.04867196083068848 seconds
Time elapsed: 0.5266127586364746 seconds
